http://brandonrose.org/clustering#Tf-idf-and-document-similarity

In [2]:
import pandas as pd
import numpy as np
import collections
import json

In [3]:
lda_result = pd.read_csv('./lda.tsv', sep='\t')
km_result = pd.read_csv('./km.tsv', sep='\t')

In [11]:
len(km_result)

400

In [16]:
km_result = pd.DataFrame([0, 6, 0, 6, 6, 6, 6, 0, 0, 6, 0, 0, 0, 6, 0, 6, 6, 6, 6, 0, 0, 0,
       0, 0, 2, 6, 6, 0, 6, 6, 2, 0, 0, 0, 0, 6, 4, 2, 6, 7, 0, 6, 6, 0,
       5, 0, 6, 6, 6, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       2, 1, 1, 1, 1, 1, 1, 1, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 6, 2, 2, 2, 2,
       6, 2, 2, 2, 2, 2, 2, 2, 6, 2, 2, 2, 2, 2, 2, 2, 2, 2, 6, 2, 2, 2,
       2, 2, 6, 2, 6, 2, 2, 2, 2, 4, 2, 2, 6, 2, 2, 2, 6, 2, 2, 2, 2, 2,
       4, 2, 4, 4, 4, 4, 0, 7, 7, 7, 4, 4, 0, 2, 4, 0, 7, 7, 4, 7, 0, 4,
       4, 2, 4, 7, 0, 7, 0, 0, 0, 4, 4, 2, 7, 0, 4, 4, 4, 0, 0, 4, 4, 0,
       4, 0, 4, 4, 4, 4, 2, 4, 5, 6, 6, 6, 5, 5, 6, 5, 5, 5, 5, 5, 6, 5,
       6, 5, 7, 5, 6, 7, 5, 0, 0, 6, 6, 6, 6, 5, 5, 5, 0, 6, 6, 6, 6, 2,
       7, 7, 5, 5, 7, 5, 6, 2, 6, 6, 7, 0, 5, 7, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       4, 0, 0, 0, 4, 4, 7, 7, 0, 7, 7, 2, 7, 7, 7, 2, 7, 7, 2, 7, 7, 2,
       7, 7, 7, 7, 7, 7, 7, 7, 0, 7, 7, 2, 7, 7, 7, 6, 7, 2, 7, 7, 7, 7,
       7, 7, 7, 7], columns=['topic']).reset_index()
km_result.columns = ['id', 'topic']

In [17]:
km_result.head()

,id,topic
0,0,0
1,1,6
2,2,0
3,3,6
4,4,6


In [18]:
def intersect(a, b):
    return len(list(set(a) & set(b)))

In [28]:
lda_docs_by_topic = lda_result.groupby('topic').agg({'id': 'unique'})
km_docs_by_topic = km_result.groupby('topic').agg({'id': 'unique'})

In [20]:
lda_km_topic_map = {} # key: km, value: lda

In [21]:
km_docs_by_topic.reset_index().id.apply(lambda x: len(x))

0    96
1    46
2    59
3    48
4    30
5    20
6    51
7    50
Name: id, dtype: int64

In [22]:
for i in range(8):
    lda_km_topic_map[i] = np.argmax([intersect(km_docs_by_topic.iloc[i].values[0], lda_docs_by_topic.iloc[j].values[0]) for j in range(8)])
    print('KM topic' + str(i+1))
    print([intersect(km_docs_by_topic.iloc[i].values[0], lda_docs_by_topic.iloc[j].values[0]) for j in range(8)])
    print('LAD topic' + str(np.argmax([intersect(km_docs_by_topic.iloc[i].values[0], lda_docs_by_topic.iloc[j].values[0]) for j in range(8)]) + 1))

KM topic1
[22, 1, 6, 9, 8, 43, 4, 3]
LAD topic6
KM topic2
[0, 0, 1, 0, 0, 0, 45, 0]
LAD topic7
KM topic3
[5, 1, 3, 6, 2, 1, 3, 38]
LAD topic8
KM topic4
[0, 30, 9, 1, 0, 0, 8, 0]
LAD topic2
KM topic5
[3, 0, 4, 2, 18, 0, 0, 3]
LAD topic5
KM topic6
[6, 0, 1, 2, 5, 4, 0, 2]
LAD topic1
KM topic7
[28, 1, 1, 6, 2, 2, 2, 9]
LAD topic1
KM topic8
[4, 4, 7, 27, 5, 0, 0, 3]
LAD topic4


In [23]:
lda_km_topic_map

{0: 5, 1: 6, 2: 7, 3: 1, 4: 4, 5: 0, 6: 0, 7: 3}

In [29]:
merged_result = pd.merge(lda_result[['id', 'document', 'topic']], km_result[['id', 'topic']], on='id', suffixes=('_lda', '_km'))
merged_result.topic_km = merged_result.topic_km.apply(lambda x: lda_km_topic_map[x])

In [30]:
merged_result.head()

,id,document,topic_lda,topic_km
0,0,Hazard Ratio (and 95% Confidence Intervals) ...,0,0
1,1,Left Ventricular and Right Ventricular Ejectio...,0,6
2,2,Partial Regression Coefficients (All Subjects...,0,0
3,3,Left Ventricular (LVEF) and Right Ventricula...,0,6
4,4,Predictors of Mortality by Multivariable Analy...,0,6


In [31]:
json_data = collections.OrderedDict()
json_data['rows'] = merged_result[['document', 'topic_lda', 'topic_km']].to_dict(orient='records')
jsonp = json.loads(json.dumps(json_data, ensure_ascii=False, indent='\t').replace('`', ''))

In [32]:
f = open('./document_table.json', 'w')
f.write(json.dumps(json_data, ensure_ascii=False, indent='\t').replace('`', ''))
f.close()